In [37]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
%matplotlib notebook

def load_orientation_data(filename):
    """
    Load the data from the CSV file
    """
    df = pd.read_csv(filename, header=None)
    df.columns = ['time', 'qw', 'qx', 'qy', 'qz']
    return df

def load_inertial_data(filename):
    """
    Load the data from the CSV file
    """
    df = pd.read_csv(filename, header=None, names=['time', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz'])
    return df
# Load the CSV file
orientation_df = load_orientation_data('../data/helmet-data/dataset-helmet9/inertialSense_qtn.csv')
inertial_df = load_inertial_data('../data/helmet-data/dataset-helmet9/inertialSense_imu.csv')

df = pd.concat([orientation_df, inertial_df.drop("time", axis=1)], axis=1)

# Only take every 100th row
df = df.iloc[::10, :]

np.float64(-0.0103024472804754)

In [11]:
inertial = pd.read_csv('../data/helmet-data/dataset-helmet9/inertialSense_imu.csv', header=None, names=['time', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz'])
inertial2 = pd.read_csv('../data/helmet-data/dataset-helmet9/vectorNav_imu.csv', header=None, names=['time', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz'])
(inertial).describe()

,time,ax,ay,az,gx,gy,gz,mx,my,mz
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,2501.998000,0.004163,-0.230764,0.016576,-1.111510,-9.128361,0.582281,0.175815,0.852244,-0.159313
std,86.603118,0.543377,2.245598,0.588593,3.178000,2.150270,3.058452,0.359481,0.094774,0.281395
min,2352.000000,-4.302342,-11.580304,-4.723484,-21.659762,-22.144312,-21.583282,-0.832271,0.078372,-0.994126
25%,2426.999000,-0.134365,-0.949808,-0.180796,-2.677997,-9.744245,-0.351486,-0.096124,0.804817,-0.342677
50%,2501.998000,0.001897,-0.012312,0.000737,-0.855455,-9.317269,0.293502,0.285614,0.871431,-0.257902
75%,2576.997000,0.160744,0.492466,0.181477,0.541802,-8.530279,1.771153,0.472140,0.907413,0.082106
max,2651.996000,3.319516,10.896223,5.307124,12.740695,6.685563,21.960497,0.957956,0.999997,0.812462


In [12]:
[i for i in range(1, 12)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [13]:
# THIS IS WRONG. WE NEED TO CONCIDER THE INERTIAL DATA, NOT THE ORIENTATION DATA.

# There are 12 datasets. They are unlabled and we need to find the dataset where the person is running.
# We can do this by analzing the variance of the quaternions. Persumably, the variance is highest when the person is running.

# We will start by loading all the datasets.
dataframes = [load_orientation_data(f'../data/helmet-data/dataset-helmet{i}/camera_qtn.csv') for i in range(1, 13)]

# Now we will calculate the variance of the quaternions for each dataset.
variances = [df.var().drop("time").sum() for df in dataframes]

# Find the index of the dataset with the highest variance
index = np.argmax(variances)
int(index + 1)





7

In [14]:
def quaternion_to_matrix(qw, qx, qy, qz):
    r00 = 1 - 2*(qy**2 + qz**2)
    r01 = 2*(qx*qy - qz*qw)
    r02 = 2*(qx*qz + qy*qw)
    r10 = 2*(qx*qy + qz*qw)
    r11 = 1 - 2*(qx**2 + qz**2)
    r12 = 2*(qy*qz - qx*qw)
    r20 = 2*(qx*qz - qy*qw)
    r21 = 2*(qy*qz + qx*qw)
    r22 = 1 - 2*(qx**2 + qy**2)
    return np.array([[r00, r01, r02], [r10, r11, r12], [r20, r21, r22]])

In [15]:
fig, ax = plt.subplots()
t = np.linspace(0, 3, 40)
g = -9.81
v0 = 12
z = g * t**2 / 2 + v0 * t

v02 = 5
z2 = g * t**2 / 2 + v02 * t

scat = ax.scatter(t[0], z[0], c="b", s=5, label=f'v0 = {v0} m/s')
line2 = ax.plot(t[0], z2[0], label=f'v0 = {v02} m/s')[0]
ax.set(xlim=[0, 3], ylim=[-4, 10], xlabel='Time [s]', ylabel='Z [m]')
ax.legend()


def update(frame):
    # for each frame, update the data stored on each artist.
    x = t[:frame]
    y = z[:frame]
    # update the scatter plot:
    data = np.stack([x, y]).T
    scat.set_offsets(data)
    # update the line plot:
    line2.set_xdata(t[:frame])
    line2.set_ydata(z2[:frame])
    return (scat, line2)


ani = FuncAnimation(fig=fig, func=update, frames=20, interval=30)

from IPython.display import HTML
HTML(ani.to_jshtml())

<IPython.core.display.Javascript object>

In [39]:
# Define vertices for a cube
cube_vertices = np.array([
    [1, 1, 1], [-1, 1, 1], [-1, -1, 1], [1, -1, 1],  # Front face
    [1, 1, -1], [-1, 1, -1], [-1, -1, -1], [1, -1, -1]  # Back face
])

# Define vertices for a smiley face on the front face of the cube
smiley_face_vertices = np.array([
    [0.5, 0.5, 1], [-0.5, 0.5, 1],  # Eyes
    [0, -0.5, 1],  # Mouth center
    [0.3, -0.6, 1], [0.1, -0.7, 1], [-0.1, -0.7, 1], [-0.3, -0.6, 1]  # Mouth corners
])

# Combine cube vertices and smiley face vertices
mech_vertices = np.vstack((cube_vertices, smiley_face_vertices))

   

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Calculate time intervals in milliseconds
time_intervals = (df['time'].diff().fillna(0) * 1000).astype(int)
# Get the average time interval
average_interval = time_intervals.mean()
print("Average interval: ", average_interval)

def update(num, data, mech_vertices, mech):
    ax.clear()
    qw, qx, qy, qz = data.iloc[num, 1:5]  # Skip the time column
    rotation_matrix = quaternion_to_matrix(qw, qx, qy, qz)
    transformed_vertices = np.dot(mech_vertices, rotation_matrix.T)
    ax.scatter(transformed_vertices[:, 0], transformed_vertices[:, 1], transformed_vertices[:, 2])
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([-2, 2])

    # Plot the accelerations
    ax.quiver(0, 0, 0, data["ax"].iloc[num], data["ay"].iloc[num], data["az"].iloc[num], color='r')
    
    elapsed_time = data.iloc[num, 0] - data.iloc[0, 0]  # Assuming the time is in the first column
    ax.text(0, 0, 0, f'Elapsed time: {elapsed_time}', transform=ax.transAxes)
    
    return mech,


# Function to get the appropriate interval for each frame
def frame_interval_gen():
    for interval in time_intervals:
        print("Getting interval: ", interval)
        yield interval

interval_gen = frame_interval_gen()

def get_next_interval(*args):
    return next(interval_gen)

ani = FuncAnimation(fig, update, frames=len(df), fargs=(df, mech_vertices, ax), repeat=True)

# Keep a reference to the animation object to prevent it from being garbage collected
#plt.show()

# To save the animation as a video file
ani.save('mech_animation.mp4', writer='ffmpeg', fps=24)

<IPython.core.display.Javascript object>

Average interval:  39.0748


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10324fbf0>>
Traceback (most recent call last):
  File "/Users/sebastian/Documents/Aalto/Savox Battery Booster/python/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
